In [4]:
import pandas as pd
import numpy as np
import glob
import os

In [6]:
os.getcwd()

'f:\\git\\donkey_car_implemented_lag'

In [60]:
csv_files = glob.glob(os.path.join("data", "model_predictions", "*tflite.csv"))
dfs = []

for csv_file in csv_files:
    df = pd.read_csv(csv_file, sep = ';')
    print(csv_file.split("\\")[-1])
    # print(csv_file.split("\\")[-1].split(".")[0][17:])
    df['model'] = csv_file.split("\\")[-1].split(".")[0][17:]
    dfs.append(df)

new_df = pd.concat(dfs)
new_df = new_df.drop("Unnamed: 0", axis=1)
# I did it wrong before, so I have to do it wrong again
# This is correct
new_df['tub'] = [x.split("/")[3][:-9] for x in new_df['tub']] 
# new_df['tub'] = [x.split("/")[3][:7] for x in new_df['tub']] 
# get over 1 values to 1, min
new_df['angles'] = [min(angle, 1) for angle in new_df['angles']]
# get under -1 values to -1, max
new_df['angles'] = [max(angle, -1) for angle in new_df['angles']]
new_df.head()

predictions_20hz_frameshifted_-1.tflite.csv
predictions_20hz_frameshifted_-2.tflite.csv
predictions_20hz_frameshifted_1.tflite.csv
predictions_20hz_frameshifted_2.tflite.csv
predictions_20hz_frameshifted_3.tflite.csv
predictions_20hz_frameshifted_4.tflite.csv
predictions_20hz_lowres.tflite.csv


,angles,frame_indexes,tub,model
0,-1.0,0,tub_89,frameshifted_-1
1,-1.0,10000,tub_89,frameshifted_-1
2,-1.0,10001,tub_89,frameshifted_-1
3,-1.0,10002,tub_89,frameshifted_-1
4,-1.0,10003,tub_89,frameshifted_-1


In [61]:
new_df.dtypes

angles           float64
frame_indexes      int64
tub               object
model             object
dtype: object

In [62]:
new_df['tub'].value_counts()

tub_89     196175
tub_175    158641
tub_168    125314
Name: tub, dtype: int64

# Get ground truth

In [93]:
ground_truth = pd.read_csv("data/model_predictions/test_tubs_df.csv", sep = ';')
ground_truth.head()

,Unnamed: 0,_index,_session_id,_timestamp_ms,behavior/label,behavior/one_hot_state_array,behavior/state,cam/image_array,user/angle,user/mode,user/throttle,tub_name,deleted_index
0,0,0,22-04-07_0,1649337993704,Straight,"[0.0, 1.0, 0.0]",1,0_cam_image_array_.jpg,0.0,user,0.490142,tub_89,True
1,1,1,22-04-07_0,1649337993713,Straight,"[0.0, 1.0, 0.0]",1,1_cam_image_array_.jpg,0.0,user,0.620000,tub_89,True
2,2,2,22-04-07_0,1649337993759,Straight,"[0.0, 1.0, 0.0]",1,2_cam_image_array_.jpg,0.0,user,0.620000,tub_89,True
3,3,3,22-04-07_0,1649337993809,Straight,"[0.0, 1.0, 0.0]",1,3_cam_image_array_.jpg,0.0,user,0.620000,tub_89,True
4,4,4,22-04-07_0,1649337993859,Straight,"[0.0, 1.0, 0.0]",1,4_cam_image_array_.jpg,0.0,user,0.650000,tub_89,True


In [94]:
ground_truth = ground_truth.drop(['Unnamed: 0', '_session_id', '_timestamp_ms', 
'behavior/label', 'behavior/one_hot_state_array', 'behavior/state', 'cam/image_array',
'user/throttle', 'user/mode'], axis = 1)
ground_truth.head()

,_index,user/angle,tub_name,deleted_index
0,0,0.0,tub_89,True
1,1,0.0,tub_89,True
2,2,0.0,tub_89,True
3,3,0.0,tub_89,True
4,4,0.0,tub_89,True


In [95]:
ground_truth['tub_name'].value_counts()

tub_89     28025
tub_175    22663
tub_168    17902
Name: tub_name, dtype: int64

In [96]:
ground_truth['_index_fs0'] = [index for index in ground_truth['_index']]
ground_truth['_index_fs1'] = [index - 1 for index in ground_truth['_index']]
ground_truth['_index_fs2'] = [index - 2 for index in ground_truth['_index']]
ground_truth['_index_fs3'] = [index - 3 for index in ground_truth['_index']]
ground_truth['_index_fs4'] = [index - 4 for index in ground_truth['_index']]
ground_truth['_index_fs-1'] = [index - (-1) for index in ground_truth['_index']]
ground_truth['_index_fs-2'] = [index - (-2) for index in ground_truth['_index']]
ground_truth.head()

,_index,user/angle,tub_name,deleted_index,_index_fs0,_index_fs1,_index_fs2,_index_fs3,_index_fs4,_index_fs-1,_index_fs-2
0,0,0.0,tub_89,True,0,-1,-2,-3,-4,1,2
1,1,0.0,tub_89,True,1,0,-1,-2,-3,2,3
2,2,0.0,tub_89,True,2,1,0,-1,-2,3,4
3,3,0.0,tub_89,True,3,2,1,0,-1,4,5
4,4,0.0,tub_89,True,4,3,2,1,0,5,6


In [74]:
new_df.head()

,angles,frame_indexes,tub,model
0,-1.0,0,tub_89,frameshifted_-1
1,-1.0,10000,tub_89,frameshifted_-1
2,-1.0,10001,tub_89,frameshifted_-1
3,-1.0,10002,tub_89,frameshifted_-1
4,-1.0,10003,tub_89,frameshifted_-1


In [76]:
new_df.model.unique()

array(['frameshifted_-1', 'frameshifted_-2', 'frameshifted_1',
       'frameshifted_2', 'frameshifted_3', 'frameshifted_4', 'lowres'],
      dtype=object)

In [99]:
baseline = pd.merge(new_df[new_df['model'] == 'lowres'], ground_truth, left_on = ["frame_indexes", "tub"], right_on = ["_index_fs0", "tub_name"])
frameshifted_1 = pd.merge(new_df[new_df['model'] == 'frameshifted_1'], ground_truth, left_on = ["frame_indexes", "tub"], right_on = ["_index_fs1", "tub_name"])
frameshifted_2 = pd.merge(new_df[new_df['model'] == 'frameshifted_2'], ground_truth, left_on = ["frame_indexes", "tub"], right_on = ["_index_fs2", "tub_name"])
frameshifted_3 = pd.merge(new_df[new_df['model'] == 'frameshifted_3'], ground_truth, left_on = ["frame_indexes", "tub"], right_on = ["_index_fs3", "tub_name"])
frameshifted_4 = pd.merge(new_df[new_df['model'] == 'frameshifted_4'], ground_truth, left_on = ["frame_indexes", "tub"], right_on = ["_index_fs4", "tub_name"])
frameshifted_min1 = pd.merge(new_df[new_df['model'] == 'frameshifted_-1'], ground_truth, left_on = ["frame_indexes", "tub"], right_on = ["_index_fs-1", "tub_name"])
frameshifted_min2 = pd.merge(new_df[new_df['model'] == 'frameshifted_-2'], ground_truth, left_on = ["frame_indexes", "tub"], right_on = ["_index_fs-2", "tub_name"])

In [100]:
# final_df = pd.merge(new_df, ground_truth, left_on = ["frame_indexes", "tub"], right_on = ["_index", "tub_name"])
final_df = pd.concat([baseline, frameshifted_1, frameshifted_2, frameshifted_3, frameshifted_4, frameshifted_min1, frameshifted_min2])

In [101]:
final_df

,angles,frame_indexes,tub,model,_index,user/angle,tub_name,deleted_index,_index_fs0,_index_fs1,_index_fs2,_index_fs3,_index_fs4,_index_fs-1,_index_fs-2
0,-0.144916,0,tub_89,lowres,0,0.000000,tub_89,True,0,-1,-2,-3,-4,1,2
1,-1.000000,10000,tub_89,lowres,10000,0.000000,tub_89,True,10000,9999,9998,9997,9996,10001,10002
2,-1.000000,10001,tub_89,lowres,10001,0.600757,tub_89,True,10001,10000,9999,9998,9997,10002,10003
3,-1.000000,10002,tub_89,lowres,10002,1.000000,tub_89,True,10002,10001,10000,9999,9998,10003,10004
4,-1.000000,10003,tub_89,lowres,10003,1.000000,tub_89,True,10003,10002,10001,10000,9999,10004,10005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68579,-0.649491,9998,tub_175,frameshifted_-2,9996,-1.000000,tub_175,True,9996,9995,9994,9993,9992,9997,9998
68580,-1.000000,9999,tub_175,frameshifted_-2,9997,-1.000000,tub_175,True,9997,9996,9995,9994,9993,9998,9999
68581,-0.872467,999,tub_175,frameshifted_-2,997,1.000000,tub_175,True,997,996,995,994,993,998,999
68582,0.219575,99,tub_175,frameshifted_-2,97,-1.000000,tub_175,True,97,96,95,94,93,98,99


In [111]:
final_df[(final_df['frame_indexes'] != final_df['_index_fs1']) & (final_df['model'] == "frameshifted_2")]

,angles,frame_indexes,tub,model,_index,user/angle,tub_name,deleted_index,_index_fs0,_index_fs1,_index_fs2,_index_fs3,_index_fs4,_index_fs-1,_index_fs-2
0,-1.000000,0,tub_89,frameshifted_2,2,0.0,tub_89,True,2,1,0,-1,-2,3,4
1,-0.850773,10000,tub_89,frameshifted_2,10002,1.0,tub_89,True,10002,10001,10000,9999,9998,10003,10004
2,-0.828374,10001,tub_89,frameshifted_2,10003,1.0,tub_89,True,10003,10002,10001,10000,9999,10004,10005
3,-0.852595,10002,tub_89,frameshifted_2,10004,1.0,tub_89,True,10004,10003,10002,10001,10000,10005,10006
4,-0.849245,10003,tub_89,frameshifted_2,10005,1.0,tub_89,True,10005,10004,10003,10002,10001,10006,10007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68579,-1.000000,9998,tub_175,frameshifted_2,10000,-1.0,tub_175,True,10000,9999,9998,9997,9996,10001,10002
68580,-1.000000,9999,tub_175,frameshifted_2,10001,-1.0,tub_175,True,10001,10000,9999,9998,9997,10002,10003
68581,-0.683691,999,tub_175,frameshifted_2,1001,1.0,tub_175,True,1001,1000,999,998,997,1002,1003
68582,-0.440430,99,tub_175,frameshifted_2,101,-1.0,tub_175,True,101,100,99,98,97,102,103


In [103]:
def bring_true_index(df, model, index_column_name):
    changed_df = df[df['model'] == model][['angles', 'frame_indexes', 'tub', 'model', 'user/angle', 'deleted_index', index_column_name]]
    changed_df.rename({index_column_name: "index_fs"}, inplace = True)
    return changed_df

In [113]:
# bring_true_index(final_df, "frameshifted_4", "_index_fs4")
# bring_true_index(final_df, "frameshifted_1", "_index_fs1")[bring_true_index(final_df, "frameshifted_1", "_index_fs1")['frame_indexes'] == 10000]
# bring_true_index(final_df, "frameshifted_1", "_index_fs1")[bring_true_index(final_df, "frameshifted_1", "_index_fs1")['frame_indexes'] != bring_true_index(final_df, "frameshifted_1", "_index_fs1")['_index_fs1']]


,angles,frame_indexes,tub,model,user/angle,deleted_index,_index_fs4
0,1.000000,0,tub_89,frameshifted_4,0.0,True,0
1,-0.315443,10000,tub_89,frameshifted_4,1.0,True,10000
2,-0.376870,10001,tub_89,frameshifted_4,1.0,True,10001
3,-0.253507,10002,tub_89,frameshifted_4,1.0,True,10002
4,-0.200435,10003,tub_89,frameshifted_4,1.0,True,10003
...,...,...,...,...,...,...,...
68573,-0.600717,9998,tub_175,frameshifted_4,-1.0,True,9998
68574,-0.575310,9999,tub_175,frameshifted_4,-1.0,True,9999
68575,-0.151150,999,tub_175,frameshifted_4,1.0,True,999
68576,-0.495554,99,tub_175,frameshifted_4,-1.0,True,99


In [82]:
final_df.to_csv("test_ground_truth_and_predictions_fixed.csv", sep = ';', index = False)